# Introduction

### Preparing the Dataset

The Dataloader loads the dataset and splits the dataset into train, test, and validation datasets. Dataloader class has the implementation of sliding window technique and stratified sampling technique.

In [ ]:
from Utils.LoadDataset import Dataloader

dataset  = Dataloader(data_path = "./Custom Dataset/Raw Data",
                  labels_path = "./Custom Dataset/labels.csv", 
                  seperator = ",", test_ratio = 0.2, validation_ratio = 0.2,
                  classnames=['running', 'walking', 'sitting', 'lying'])

### Defining the Model

The model developed for HAR in bangle.js is a simple convolutional neural network with two convolution layers, one max-pool layer, and a single dense layer. The first convolutional layer has fourteen kernels, and the number of kernels used in the second convolutional layer is equal to the number of class names.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

tf.random.set_seed(1)

model = keras.Sequential()
model.add(keras.Input(shape = (50, 1, 3), name = "accelartion"))
model.add(keras.layers.Conv2D(filters = 14, kernel_size = (5, 1), padding = "valid", activation = "relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(filters = len(dataset.classnames), kernel_size = (5, 1), padding = "same", activation = "relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.MaxPool2D((46, 1)))
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(len(dataset.classnames), use_bias=False))

model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

### Training the Model

Sparse categorical cross-entropy is used as the loss function to measure the loss, and the Adam optimizer is used as the optimization algorithm. The model trained on the training set and a validation dataset is used to fine-tune the model.

In [ ]:
model.fit(dataset.train.x, dataset.train.y,
                    batch_size=64,
                    epochs=200,
                    verbose=1,
                    validation_data=(dataset.validation.x, dataset.validation.y))

### Optimizing the Model

We optimize it using TensorFlow lite, reducing its size without losing much accuracy. The export class has the converter implementation. It also parses the tflite flatbuffer file and checks the operator compatibility for the microcontrollers to ensure that the TensorFlow Lite supports all the operators included in the current model.

In [ ]:
from Utils.Optimization import OptimizeModel

tflite_converted_model = OptimizeModel(model, dataset, quantize = True, assertion = True)

To check the size of the model,

In [ ]:
print(tflite_converted_model.size_report())

Before deploying the file, a base64 encoding is performed on the tflite flatbuffer file to avoid misinterpretation of data when transferred between different channels.

In [ ]:
tflite_converted_model.base64()

### Model Deployment

To deploy the model on the bangle.js. First, we need to require the "Storage" module and transfer the model using the write method. Then, connect to the watch via Bluetooth using the espruino web-based IDE and paste the below cell's output on the left-hand side pane of the espruino based IDE and click enter.

In [ ]:
print(f'require("Storage").write(".tfmodel",atob("{tflite_converted_model.base64()}"));')

We also need the names of the activities to classify and display activity information on the watch. So we transfer the names of activities as a string.

In [ ]:
print(f'require("Storage").write(".tfnames", "running,walking,sitting,lying");')